In [1]:
import itertools

import pandas as pd
import geopandas as gpd

/Users/aolsen/opt/anaconda3/envs/geo_env_2/lib/python3.10/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/var/folders/x8/3_n775lx7zq1nv3mddc7vmlh0000gp/T/ipykernel_49401/1957567560.py:4: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Sha

In [3]:
bayareafips_full = {'06001': 'Alameda County', '06013': 'Contra Costa County', '06041': 'Marin County', '06055': 'Napa County',
                    '06075': 'San Francisco County', '06081': 'San Mateo County', '06085': 'Santa Clara County',
                    '06097': 'Sonoma County', '06095': 'Solano County'}

bayareareg = dict([(k,'ABAG/MTC') for k in bayareafips_full])


In [4]:
# for halo counties, rely on the regionalization in
# http://www.cdss.ca.gov/research/res/pdf/multireports/RegionsofCalifornia.pdf


halofips = {'06077': 'San Joaquin', '06099': 'Stanislaus', '06067': 'Sacramento', '06047': 'Merced',
            '06113': 'Yolo', '06115': 'Yuba', '06101': 'Sutter', '06017': 'El Dorado', '06061': 'Placer',
            '06087': 'Santa Cruz', '06069': 'San Benito', '06053': 'Monterey',
            '06033': 'Lake', '06045': 'Mendocino'}

halotype = {'06077': 'Halo Southern Farm', '06047': 'Halo Southern Farm', '06033': 'Halo North and Mountains', '06045': 'Mendocino',
            '06045': 'Halo North and Mountains',
            '06099': 'Halo Southern Farm', '06067': 'Halo Central Valley',
            '06113': 'Halo Central Valley', '06115': 'Halo Central Valley', '06101': 'Halo Central Valley',
            '06017': 'Halo Central Valley', '06061': 'Halo Central Valley',
            '06087': 'Halo Southern Coast', '06069': 'Halo Southern Farm', '06053': 'Halo Southern Coast'}

bayareareg_w_halo = bayareareg | halotype
bayareareg_w_halo

{'06001': 'ABAG/MTC',
 '06013': 'ABAG/MTC',
 '06041': 'ABAG/MTC',
 '06055': 'ABAG/MTC',
 '06075': 'ABAG/MTC',
 '06081': 'ABAG/MTC',
 '06085': 'ABAG/MTC',
 '06097': 'ABAG/MTC',
 '06095': 'ABAG/MTC',
 '06077': 'Halo Southern Farm',
 '06047': 'Halo Southern Farm',
 '06033': 'Halo North and Mountains',
 '06045': 'Halo North and Mountains',
 '06099': 'Halo Southern Farm',
 '06067': 'Halo Central Valley',
 '06113': 'Halo Central Valley',
 '06115': 'Halo Central Valley',
 '06101': 'Halo Central Valley',
 '06017': 'Halo Central Valley',
 '06061': 'Halo Central Valley',
 '06087': 'Halo Southern Coast',
 '06069': 'Halo Southern Farm',
 '06053': 'Halo Southern Coast'}

In [5]:
cog_regions = {'06001': 'ABAG/MTC',
               '06013': 'ABAG/MTC',
               '06041': 'ABAG/MTC',
               '06053': 'AMBAG',
               '06055': 'ABAG/MTC',
               '06069': 'AMBAG',
               '06075': 'ABAG/MTC',
               '06073': 'SANDAG',
               '06081': 'ABAG/MTC',
               '06085': 'ABAG/MTC',
               '06087': 'AMBAG',
               '06095': 'ABAG/MTC',
               '06097': 'ABAG/MTC',
               '06077': 'SJCOG',
               '06099': 'STANCOG',
               '06047': 'MCAG',
               '06025': 'SCAG',  # imperial
               '06037': 'SCAG',   # LA
               '06071': 'SCAG',  # san bernadino
               '06059': 'SCAG',  # orange
               '06065': 'SCAG',  # riverside
               '06111': 'SCAG',  # ventura
               '06067': 'SACOG',  # 'yolo'
               '06101': 'SACOG',  # 'sutter'
               '06113': 'SACOG',  # 'yolo'
               '06115': 'SACOG'
               }  # 'yuba' }
# cog_regions.update(halotype)
cog_regions

{'06001': 'ABAG/MTC',
 '06013': 'ABAG/MTC',
 '06041': 'ABAG/MTC',
 '06053': 'AMBAG',
 '06055': 'ABAG/MTC',
 '06069': 'AMBAG',
 '06075': 'ABAG/MTC',
 '06073': 'SANDAG',
 '06081': 'ABAG/MTC',
 '06085': 'ABAG/MTC',
 '06087': 'AMBAG',
 '06095': 'ABAG/MTC',
 '06097': 'ABAG/MTC',
 '06077': 'SJCOG',
 '06099': 'STANCOG',
 '06047': 'MCAG',
 '06025': 'SCAG',
 '06037': 'SCAG',
 '06071': 'SCAG',
 '06059': 'SCAG',
 '06065': 'SCAG',
 '06111': 'SCAG',
 '06067': 'SACOG',
 '06101': 'SACOG',
 '06113': 'SACOG',
 '06115': 'SACOG'}

In [8]:
state_fips=pd.read_csv('/Users/aolsen/Dropbox/Documents/Data/_Census/lookups/statefips.txt',dtype=str)
ca_geos = gpd.read_file('/Users/aolsen/Dropbox/Documents/Data/GIS/ca_counties.shp')
#set(ca_geos.GEOID)-set(cog_regions.keys())
ca_geos.head()

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,NAMELSAD,STUSPS,STATE_NAME,LSAD,ALAND,AWATER,macroreg,cogreg,geometry
0,06,037,00277283,0500000US06037,06037,Los Angeles,Los Angeles County,CA,California,06,10513491099,1787501506,Southern California,SCAG,"MULTIPOLYGON (((129777.289 -503034.640, 130301..."
1,06,023,01681908,0500000US06023,06023,Humboldt,Humboldt County,CA,California,06,9241565229,1253726036,North Coast,None,"POLYGON ((-373865.773 278378.334, -372754.945 ..."
2,06,097,01657246,0500000US06097,06097,Sonoma,Sonoma County,CA,California,06,4080862341,498084088,Bay Area,ABAG,"POLYGON ((-306588.058 89313.140, -306210.097 8..."
3,06,079,00277304,0500000US06079,06079,San Luis Obispo,San Luis Obispo County,CA,California,06,8549172832,815618272,Central Coast,None,"POLYGON ((-121549.192 -246008.590, -112398.145..."
4,06,089,01682610,0500000US06089,06089,Shasta,Shasta County,CA,California,06,9778551576,186157964,Shasta Cascades,None,"POLYGON ((-260782.247 258918.819, -260645.840 ..."


In [9]:
county_class = pd.read_excel(
    '../data/Rosetta_County.xlsx', index_col=[0, 1, 2, 3], engine='openpyxl')
county_class.index = county_class.index.set_names(
    ['CBSA_CODE', 'CBSA_Title', 'STCOUNTY', 'county_name'])
county_class = county_class.reset_index()
county_class = county_class.rename(columns={'Brookings': 'Bill Frey / Brookings Inst.',
                                            'Kolko': 'Jed Kolko / Indeed',
                                            'Yonder': 'Bill Bishop / Daily Yonder'})

for klass in ['Bill Frey / Brookings Inst.', 'Jed Kolko / Indeed', 'Bill Bishop / Daily Yonder']:
    county_class[klass] = county_class[klass].str.replace(' Counties', '').str.replace(
        'Rural Adjacent to Large', 'Rrl adj. to lg.')  # .apply(lambda x: '\n'.join(x))
county_class.STCOUNTY = county_class.STCOUNTY.apply(
    lambda x: '{:0>5}'.format(x))
county_class.CBSA_CODE = county_class.CBSA_CODE.apply(
    lambda x: '{:0>5}'.format(x))

klass_order = {'Bill Frey / Brookings Inst.': ['1-Urban Core', u'2-Mature Suburb', u'3-Emerging Suburb', u'4-Exurb',
                                               u'NonMetro', 'Rest of Counties'],
               'Jed Kolko / Indeed': [u'1-Urban', u'2-SuburbanHigh', u'3-SuburbanLow', 'Rest of Counties'],
               'Bill Bishop / Daily Yonder': [u'1-Central counties', u'2-Suburban', u'3-Exurban',
                                              u'4-Rrl adj. to lg. MSA', 'Rest of Counties']}

county_class['state'] = county_class.STCOUNTY.str.slice(
    0, 2).map(state_fips.set_index('FIPS').Name)
county_class['state_county'] = county_class.state+', '+county_class.county_name
county_class.head()

/Users/aolsen/opt/anaconda3/envs/geo_env_2/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


,CBSA_CODE,CBSA_Title,STCOUNTY,county_name,Bill Frey / Brookings Inst.,Jed Kolko / Indeed,Bill Bishop / Daily Yonder,Population,Density,state,state_county
0,12060,"Atlanta-Sandy Springs-Marietta, GA",13013,Barrow County,4-Exurb,3-SuburbanLow,2-Suburban,80834,188.10490,Georgia,"Georgia, Barrow County"
1,12060,"Atlanta-Sandy Springs-Marietta, GA",13015,Bartow County,3-Emerging Suburb,3-SuburbanLow,2-Suburban,106378,169.13420,Georgia,"Georgia, Bartow County"
2,12060,"Atlanta-Sandy Springs-Marietta, GA",13035,Butts County,4-Exurb,3-SuburbanLow,3-Exurban,24192,50.40048,Georgia,"Georgia, Butts County"
3,12060,"Atlanta-Sandy Springs-Marietta, GA",13045,Carroll County,4-Exurb,3-SuburbanLow,2-Suburban,117997,143.57570,Georgia,"Georgia, Carroll County"
4,12060,"Atlanta-Sandy Springs-Marietta, GA",13057,Cherokee County,2-Mature Suburb,3-SuburbanLow,2-Suburban,253914,417.50190,Georgia,"Georgia, Cherokee County"


# Load and combine data

In [30]:
popest2010s = pd.read_csv(
    '../data/co-est2020-alldata.csv', encoding='windows-1252')
popest2020s = pd.read_csv(
    '../data/co-est2022-alldata.csv')

popest2020s = popest2020s[popest2020s.columns.difference(popest2020s.filter(regex='2020').columns)]

index_cols = ['SUMLEV', 'REGION', 'DIVISION',
              'STATE', 'COUNTY', 'STNAME', 'CTYNAME']

popest = pd.concat([popest2010s.set_index(index_cols),
                   popest2020s.set_index(index_cols)], axis=1).reset_index()

popest['STCOUNTY'] = popest.STATE.apply(lambda x: f'{x:02d}')+popest.COUNTY.apply(lambda x: f'{x:03d}')
popest['county_name'] = popest.CTYNAME.str.replace(' County','')

popest = popest[popest.SUMLEV==50]
popest['region'] = popest.STCOUNTY.map(cog_regions)
popest['region_2'] = popest.STCOUNTY.map(bayareareg_w_halo)


popest.query('STCOUNTY=="06085"').filter(regex='BIRTHS')
popest['macroreg']=popest.STCOUNTY.map(ca_geos.set_index('GEOID').macroreg)

## Add county classification

In [31]:
popest_class = popest.merge(county_class,on=['STCOUNTY'],how='left')
popest_class.update(popest_class[[ 'Bill Frey / Brookings Inst.',
       'Jed Kolko / Indeed', 'Bill Bishop / Daily Yonder']].fillna('NonMetro'))

In [32]:
popest_class.query('STATE==6').groupby(['Bill Frey / Brookings Inst.']).sum().filter(regex='POPE').T#.plot(kind='bar')

/var/folders/x8/3_n775lx7zq1nv3mddc7vmlh0000gp/T/ipykernel_49401/3266550364.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  popest_class.query('STATE==6').groupby(['Bill Frey / Brookings Inst.']).sum().filter(regex='POPE').T#.plot(kind='bar')


Bill Frey / Brookings Inst.,1-Urban Core,2-Mature Suburb,3-Emerging Suburb,4-Exurb,NonMetro
POPESTIMATE2010,18715004.0,9220973.0,531168.0,55544.0,8796861.0
POPESTIMATE2011,18876683.0,9322695.0,537286.0,56094.0,8843553.0
POPESTIMATE2012,19047540.0,9414463.0,540882.0,56704.0,8884962.0
POPESTIMATE2013,19220088.0,9500967.0,546944.0,57320.0,8928449.0
POPESTIMATE2014,19376132.0,9606490.0,552398.0,57891.0,8993795.0
POPESTIMATE2015,19529141.0,9706344.0,557497.0,58276.0,9053038.0
POPESTIMATE2016,19614581.0,9802162.0,565073.0,59270.0,9108100.0
POPESTIMATE2017,19650351.0,9888353.0,574141.0,60190.0,9164750.0
POPESTIMATE2018,19634619.0,9961054.0,583660.0,61415.0,9196715.0
POPESTIMATE2019,19569795.0,10004071.0,591975.0,62803.0,9208966.0


In [78]:
popest.query('region=="ABAG/MTC"').filter(regex='DOMES|NET').sum()

DOMESTICMIG2010      -3399.000000
DOMESTICMIG2011       4311.000000
DOMESTICMIG2012      14848.000000
DOMESTICMIG2013      17933.000000
DOMESTICMIG2014      11508.000000
DOMESTICMIG2015      -2201.000000
DOMESTICMIG2016     -30839.000000
DOMESTICMIG2017     -49971.000000
DOMESTICMIG2018     -55400.000000
DOMESTICMIG2019     -66631.000000
DOMESTICMIG2020     -86461.000000
NETMIG2010            3722.000000
NETMIG2011           41778.000000
NETMIG2012           51939.000000
NETMIG2013           60729.000000
NETMIG2014           60190.000000
NETMIG2015           54558.000000
NETMIG2016           20498.000000
NETMIG2017           -4604.000000
NETMIG2018          -22320.000000
NETMIG2019          -45454.000000
NETMIG2020          -69481.000000
RDOMESTICMIG2011        12.581910
RDOMESTICMIG2012        17.845221
RDOMESTICMIG2013        28.733178
RDOMESTICMIG2014        22.671559
RDOMESTICMIG2015         2.441124
RDOMESTICMIG2016       -23.671226
RDOMESTICMIG2017       -44.786449
RDOMESTICMIG20

# Get subsets

In [33]:
# subset to counties in the cog_regions mapping dict
capop = popest[popest.STCOUNTY.isin(cog_regions)]
cacogpop = popest[popest.region.notna()]

# subset to macro region from shapefile
camacropop = popest[popest.macroreg.notna()]

# region_2 is the vicinity region categorization
cabayareapop = popest[popest.region_2.notna()]


In [34]:
# bay area subset
bayareapop = popest.query('STCOUNTY.isin(@bayareafips_full.keys())')#.set_index(['CTYNAME'])

# Get long form data

In [35]:
# these are the demographic variables we need, or the stubs of them
component_concepts = ['BIRTHS', 'DEATHS',
                      'INTERNATIONALMIG', 'DOMESTICMIG', 'RESIDUAL']

# these are the years we want to keep
years = map(lambda x: str(x), range(2011, 2023))

# blend the two togehter
components = list(map(lambda x: ''.join(x), list(
    itertools.product(*[component_concepts, years]))))
components

['BIRTHS2011',
 'BIRTHS2012',
 'BIRTHS2013',
 'BIRTHS2014',
 'BIRTHS2015',
 'BIRTHS2016',
 'BIRTHS2017',
 'BIRTHS2018',
 'BIRTHS2019',
 'BIRTHS2020',
 'BIRTHS2021',
 'BIRTHS2022',
 'DEATHS2011',
 'DEATHS2012',
 'DEATHS2013',
 'DEATHS2014',
 'DEATHS2015',
 'DEATHS2016',
 'DEATHS2017',
 'DEATHS2018',
 'DEATHS2019',
 'DEATHS2020',
 'DEATHS2021',
 'DEATHS2022',
 'INTERNATIONALMIG2011',
 'INTERNATIONALMIG2012',
 'INTERNATIONALMIG2013',
 'INTERNATIONALMIG2014',
 'INTERNATIONALMIG2015',
 'INTERNATIONALMIG2016',
 'INTERNATIONALMIG2017',
 'INTERNATIONALMIG2018',
 'INTERNATIONALMIG2019',
 'INTERNATIONALMIG2020',
 'INTERNATIONALMIG2021',
 'INTERNATIONALMIG2022',
 'DOMESTICMIG2011',
 'DOMESTICMIG2012',
 'DOMESTICMIG2013',
 'DOMESTICMIG2014',
 'DOMESTICMIG2015',
 'DOMESTICMIG2016',
 'DOMESTICMIG2017',
 'DOMESTICMIG2018',
 'DOMESTICMIG2019',
 'DOMESTICMIG2020',
 'DOMESTICMIG2021',
 'DOMESTICMIG2022',
 'RESIDUAL2011',
 'RESIDUAL2012',
 'RESIDUAL2013',
 'RESIDUAL2014',
 'RESIDUAL2015',
 'RESIDUAL2016'

In [36]:
index_cols = ['SUMLEV', 'REGION', 'DIVISION', 'STATE', 'COUNTY', 'STNAME', 'CTYNAME', 'region','region_2',
              'macroreg', 'county_name', 'STCOUNTY']


In [87]:
# Long form data
stack_var_name = 'level_%s'%len(index_cols)

popest_long = popest.set_index(index_cols).stack().reset_index(name='value')
popest_long['year'] = popest_long[stack_var_name].str.extract('(\d{4})',expand=False)
popest_long['variable'] = popest_long[stack_var_name].str.replace(
        '(\d{4})', '')
popest_long = popest_long.drop([stack_var_name],axis=1)
popest_long = popest_long.dropna(subset=['macroreg'])

# switch sign of deaths - they should be counted as negatives
popest_long.loc[popest_long.variable=='DEATHS','value'] *=-1
popest_long.loc[popest_long.variable=='DEATHS','value']

/var/folders/x8/3_n775lx7zq1nv3mddc7vmlh0000gp/T/ipykernel_49401/2713759960.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  popest_long['variable'] = popest_long[stack_var_name].str.replace(


38370   -2152.0
38371   -9075.0
38372   -9204.0
38373   -9628.0
38374   -9252.0
          ...  
50063    -697.0
50064    -624.0
50065    -604.0
50195    -808.0
50196    -744.0
Name: value, Length: 754, dtype: object

In [88]:

popest_long.query('variable.isin(@component_concepts) & STCOUNTY.isin(@bayareafips_full)').groupby(['region', 'variable', 'year']).value.sum().unstack('year').to_clipboard()

In [89]:
# Bay Area county level summary
bayareapop_components_long = popest_long.query('variable.isin(@component_concepts) & STCOUNTY.isin(@bayareafips_full)').groupby(['CTYNAME', 'variable', 'year']).value.sum()
bayareapop_components_long

CTYNAME         variable  year
Alameda County  BIRTHS    2010     4876.0
                          2011    19222.0
                          2012    18978.0
                          2013    19471.0
                          2014    19494.0
                                   ...   
Sonoma County   RESIDUAL  2018        8.0
                          2019       -3.0
                          2020       10.0
                          2021      102.0
                          2022     -261.0
Name: value, Length: 585, dtype: object

In [90]:
# CA Macroregions
camacropop_components_long = popest_long.query('variable.isin(@component_concepts)').groupby(['macroreg', 'variable', 'year']).value.sum()
camacropop_components_long.to_csv('../output/camacropop_components_long_2020_2022.csv')
camacropop_components_long


macroreg             variable  year
Bay Area             BIRTHS    2010    21745.0
                               2011    87984.0
                               2012    86094.0
                               2013    88005.0
                               2014    87212.0
                                        ...   
Southern California  RESIDUAL  2018      -91.0
                               2019      210.0
                               2020      555.0
                               2021    -3174.0
                               2022    -3524.0
Name: value, Length: 650, dtype: object

In [91]:
# CA misc COG regions
cacogpop_components_long = popest_long.query('variable.isin(@component_concepts)').groupby(['region', 'variable', 'year']).value.sum()
cacogpop_components_long.to_csv('../output/cacogpop_components_long_2020_2022.csv')
cacogpop_components_long

region    variable  year
ABAG/MTC  BIRTHS    2010    21745.0
                    2011    87984.0
                    2012    86094.0
                    2013    88005.0
                    2014    87212.0
                             ...   
STANCOG   RESIDUAL  2018      -16.0
                    2019       19.0
                    2020       32.0
                    2021      286.0
                    2022      -14.0
Name: value, Length: 520, dtype: object

In [93]:
# Bay Area with Halo regions
bayarea2pop_components_long = popest_long.query('variable.isin(@component_concepts)').groupby(['region_2', 'variable', 'year']).value.sum()
bayarea2pop_components_long.to_csv('../output/bayareahalopop_components_long_2020_2022.csv')
bayarea2pop_components_long

region_2            variable  year
ABAG/MTC            BIRTHS    2010    21745.0
                              2011    87984.0
                              2012    86094.0
                              2013    88005.0
                              2014    87212.0
                                       ...   
Halo Southern Farm  RESIDUAL  2018      -30.0
                              2019       28.0
                              2020       83.0
                              2021      750.0
                              2022     -315.0
Name: value, Length: 325, dtype: object

# Sandboxed

In [52]:
def wide_to_long(df, index_level=['CTYNAME'], summary_level=['CTYNAME']):

    ix_len = len(index_level)
    
    region_county_string = df.groupby(summary_level).county_name.apply(lambda x: ', '.join(x))
    
    pop_components_long = df.set_index(index_level)[components].stack().groupby(
        level=summary_level+[ix_len]).sum()
    
    #pop_components_long['counties'] = region_county_string
    pop_components_long = pop_components_long.reset_index(name='value')
    
    pop_components_long['counties'] = pop_components_long[summary_level].iloc[:,0].map(region_county_string)
    pop_components_long['year'] = pop_components_long.level_1.str.extract(
        '(\d{4})', expand=False)
    pop_components_long['variable'] = pop_components_long.level_1.str.replace(
        '(\d{4})', '')
    pop_components_long.loc[pop_components_long.variable == 'DEATHS',
                            'value'] = pop_components_long.loc[pop_components_long.variable == 'DEATHS', 'value'] * -1
    return pop_components_long


bayareapop_components_long = wide_to_long(bayareapop,index_level=['CTYNAME'], summary_level=['CTYNAME'])
bayareapop_components_long

/var/folders/x8/3_n775lx7zq1nv3mddc7vmlh0000gp/T/ipykernel_49401/4015293898.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  pop_components_long['variable'] = pop_components_long.level_1.str.replace(


,CTYNAME,level_1,value,counties,year,variable
0,Alameda County,BIRTHS2011,19222.0,Alameda,2011,BIRTHS
1,Alameda County,BIRTHS2012,18978.0,Alameda,2012,BIRTHS
2,Alameda County,BIRTHS2013,19471.0,Alameda,2013,BIRTHS
3,Alameda County,BIRTHS2014,19494.0,Alameda,2014,BIRTHS
4,Alameda County,BIRTHS2015,19494.0,Alameda,2015,BIRTHS
...,...,...,...,...,...,...
535,Sonoma County,RESIDUAL2018,8.0,Sonoma,2018,RESIDUAL
536,Sonoma County,RESIDUAL2019,-3.0,Sonoma,2019,RESIDUAL
537,Sonoma County,RESIDUAL2020,10.0,Sonoma,2020,RESIDUAL
538,Sonoma County,RESIDUAL2021,102.0,Sonoma,2021,RESIDUAL
